In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost
from xgboost import XGBClassifier

In [2]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        
        log_model = LogisticRegression(solver='lbfgs')
        RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        # NB = GaussianNB()
        DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
        XGB= XGBClassifier(objective='binary:logistic',max_depth= 5,learning_rate=0.1,subsample=1,n_estimators=100)
        #svc_model = SVC(kernel = 'linear', random_state = 0)
        #knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        rfemodellist=[log_model,RF,DT,XGB] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(i, n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
        return rfelist
    

def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.2, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        #sc = StandardScaler()
        #X_train = sc.fit_transform(X_train)
        #X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def Decision(X_train,y_train,X_test):
        
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

def Xgboost(X_train,y_train,X_test):
        import xgboost
        from xgboost import XGBClassifier
        from collections import Counter
        counter = Counter(y_train)  # Count class occurrences
        ratio = counter[0] / counter[1]
        classifier = XGBClassifier(objective='binary:logistic',max_depth= 5,learning_rate=0.1,subsample=1,n_estimators=100) #,scale_pos_weight=ratio)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataset= pd.read_csv("Preprocessed_dataset.csv")
dataset1=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [5]:
indep_x= dataset1.drop('Conversion',axis=1)
dep_y=dataset1['Conversion']

In [8]:
rfelist=rfeFeature(indep_x,dep_y,10)
rfelist

LogisticRegression()
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
DecisionTreeClassifier(max_features='sqrt', random_state=0)
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


[array([[0.04391851, 0.08803141, 2.39901653, ..., 0.        , 0.        ,
         0.        ],
        [0.15572507, 0.18272468, 2.91713775, ..., 0.        , 0.        ,
         0.        ],
        [0.27749037, 0.07642272, 8.2236191 , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.05652592, 0.13382612, 2.85324058, ..., 1.        , 0.        ,
         0.        ],
        [0.02396129, 0.13838618, 1.00296447, ..., 0.        , 0.        ,
         0.        ],
        [0.1856701 , 0.05722808, 6.96473936, ..., 0.        , 0.        ,
         0.        ]]),
 array([[1.36912000e+05, 6.49787007e+03, 4.39185107e-02, ...,
         6.00000000e+00, 4.00000000e+00, 6.88000000e+02],
        [4.17600000e+04, 3.89866861e+03, 1.55725071e-01, ...,
         2.00000000e+00, 2.00000000e+00, 3.45900000e+03],
        [8.84560000e+04, 1.54642960e+03, 2.77490369e-01, ...,
         1.10000000e+01, 8.00000000e+00, 2.33700000e+03],
        ...,
        [1.25471000e+05, 4.60953464e

In [7]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=Xgboost(X_train,y_train,X_test)
    print(cm)
    print(report)
    

[[  19  175]
 [   1 1405]]
              precision    recall  f1-score   support

           0       0.95      0.10      0.18       194
           1       0.89      1.00      0.94      1406

    accuracy                           0.89      1600
   macro avg       0.92      0.55      0.56      1600
weighted avg       0.90      0.89      0.85      1600

[[  70  124]
 [ 137 1269]]
              precision    recall  f1-score   support

           0       0.34      0.36      0.35       194
           1       0.91      0.90      0.91      1406

    accuracy                           0.84      1600
   macro avg       0.62      0.63      0.63      1600
weighted avg       0.84      0.84      0.84      1600

[[  57  137]
 [  29 1377]]
              precision    recall  f1-score   support

           0       0.66      0.29      0.41       194
           1       0.91      0.98      0.94      1406

    accuracy                           0.90      1600
   macro avg       0.79      0.64      0.68    

In [9]:
#10
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)
    print(cm)
    print(report)
    
    classifier,Accuracy,report,X_test,y_test,cm=Xgboost(X_train,y_train,X_test)
    print(cm)
    print(report)

[[  15  179]
 [   3 1403]]
              precision    recall  f1-score   support

           0       0.83      0.08      0.14       194
           1       0.89      1.00      0.94      1406

    accuracy                           0.89      1600
   macro avg       0.86      0.54      0.54      1600
weighted avg       0.88      0.89      0.84      1600

[[  61  133]
 [ 132 1274]]
              precision    recall  f1-score   support

           0       0.32      0.31      0.32       194
           1       0.91      0.91      0.91      1406

    accuracy                           0.83      1600
   macro avg       0.61      0.61      0.61      1600
weighted avg       0.83      0.83      0.83      1600

[[  55  139]
 [  51 1355]]
              precision    recall  f1-score   support

           0       0.52      0.28      0.37       194
           1       0.91      0.96      0.93      1406

    accuracy                           0.88      1600
   macro avg       0.71      0.62      0.65    